In [ ]:
!git clone https://github.com/gmonteiroc21/desafio_bookbridge.git

In [ ]:
%cd desafio_bookbridge

###1- Importação e Limpeza

In [ ]:
#importação do dataset seguindo as instruções na pagina

import kagglehub

# Download latest version
data_path = kagglehub.dataset_download("pypiahmad/goodreads-book-reviews1")

print("Path to dataset files:", data_path)

In [17]:
import pandas as pd
import json
import os
import sys
sys.path.append(os.path.abspath("scripts"))
from scripts.manipulating_data import *
from scripts.data_analysis import *
from scripts.data_preparation import *
from scripts.recommender import *

file_path = "/root/.cache/kagglehub/datasets/pypiahmad/goodreads-book-reviews1/versions/1/"


In [ ]:
#transforma as interações dos usuarios em um dataframe
interactions_df = pd.read_csv(file_path+'goodreads_interactions.csv', nrows=20000000) #limita em  2M de linhas
print(interactions_df.head(1))

In [ ]:
#transforma as informações dos livros em um dataframe
books_df = select_books(file_path+'goodreads_books.json')
print(books_df)


In [ ]:
print(books_df)


In [ ]:
valid_books = interactions_df['book_id'].isin(books_df['book_id'])
interactions_read = interactions_df[(interactions_df['is_read'] == 1) & valid_books]

user_total_books = interactions_read.groupby('user_id').size().reset_index(name='user_total_books')

print(user_total_books)

tratamento do *books_df*

In [ ]:
first_df = clean_empty_rows(books_df, 'book_id')

'''books_df = duplicate_size(file_path+'goodreads_books.json', books_df, colunas_desejadas)'''

books_df['average_rating'] = pd.to_numeric(first_df['average_rating'], errors='coerce')

print(books_df.head(1))

In [ ]:
#transforma as informações dos autores em um dataframe
authors_df = pd.read_json(file_path+'goodreads_book_authors.json', lines=True)
print(authors_df.head(1))

In [ ]:
#transforma as informações de generos em um dataframe
genres_df = pd.read_json(file_path+'goodreads_book_genres_initial.json', lines=True)
print(genres_df.head(10))


In [ ]:
books_df = books_df.merge(
    genres_df[['book_id', 'genres']],  # Seleciona apenas 'book_id' e 'genres'
    on='book_id',  # A chave de junção
    how='left'  # Mantém todos os livros, mesmo aqueles sem gênero
)

### 2 - Análise exploratória e preparação dos dados para modelagem

In [8]:
unique_genres = ['history, historical fiction, biography', 'fiction', 'fantasy, paranormal'
 ,'mystery, thriller, crime', 'poetry', 'romance', 'non-fiction', 'children',
 'young-adult', 'comics, graphic']


In [ ]:
genre_stats_df = genre_stats(genres_df, books_df, unique_genres)
print(genre_stats_df)

In [ ]:
top_genre_per_user = calculate_top_genre_per_user(interactions_df, genres_df,books_df)
print(top_genre_per_user.head(5))

In [ ]:
print(top_genre_per_user.head(5))

In [ ]:
user_literary_age = calculate_user_literary_age(interactions_df, books_df)
user_literary_age

### Sugestão de livro e probabilidade de boa avaliação

In [ ]:
user_id = int(input("Digite o seu ID de usuário: "))


best_book, user_genre = recommend_books(user_id, top_genre_per_user, genre_stats_df)
print(f'Levando em consideração que seu gênero mais lido é o: {user_genre}\nVocê deve curtir o {best_book}')


In [ ]:
book_id = int(input('Digite o ID do livro escolhido: '))
book_genre = get_book_genre(book_id, genres_df)
genre_count = count_similarbooks()
#affinity = calculate_affinity(user_id, book_genre, book_period, user_genre_count_df, user_time_period_count_df)
#print(f'Afinidade do usuário {user_id} para o livro: {affinity}')